<a href="https://colab.research.google.com/github/kevmanning/DS-Unit-2-Applied-Modeling/blob/master/DS_232_assignment_Kevin_Manning_dspt10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [ ]:
# Change column names: replace spaces with underscores
df.columns= df.columns.str.replace(' ', '_')
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [ ]:
# Get Pandas Profiling Report
import pandas_profiling

In [ ]:
df.profile_report()

#profile = ProfileReport(df, minimal=True).to_notebook_iframe()

Summarize dataset:   0%|          | 0/34 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)
df.sample(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
16219,1,MURRAY HILL,13 CONDOS - ELEVATOR APARTMENTS,2,868,1387,NaN,R4,"80 PARK AVENUE, 16L",16L,10016.0,1.0,0.0,1.0,"17,280",555.0,1956.0,2,R4,$ - 0,03/22/2019
2215,3,BENSONHURST,02 TWO FAMILY DWELLINGS,1,6331,40,NaN,B3,2280 83RD STREET,NaN,11214.0,2.0,0.0,2.0,"3,000",2000.0,1920.0,1,B3,"$ 1,390,000",01/14/2019
2214,3,BENSONHURST,01 ONE FAMILY DWELLINGS,1,5569,47,NaN,A2,1865 68TH STREET,NaN,11204.0,1.0,0.0,1.0,"2,000",2048.0,1915.0,1,A2,"$ 780,000",01/14/2019


In [ ]:
df= df[df.NEIGHBORHOOD == 'TRIBECA']

In [ ]:
df.shape

(146, 21)

In [ ]:
# Q. What's the date range of these property sales in Tribeca?
df.SALE_DATE.min(), df.SALE_DATE.max()

('01/03/2019', '04/30/2019')

In [ ]:
df.SALE_PRICE

220       $   2,800,000
763       $   2,650,000
996             $   - 0
1276      $   1,005,000
1542     $   12,950,000
              ...      
22221     $   5,761,259
22732     $   2,600,000
22733       $   605,000
22897       $   960,000
22898       $   975,000
Name: SALE_PRICE, Length: 146, dtype: object

In [ ]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers
df['SALE_PRICE']= df['SALE_PRICE'].map(lambda x: x.lstrip('$   -').replace(',','')).astype(int)

In [ ]:
df['SALE_PRICE']

220       2800000
763       2650000
996             0
1276      1005000
1542     12950000
           ...   
22221     5761259
22732     2600000
22733      605000
22897      960000
22898      975000
Name: SALE_PRICE, Length: 146, dtype: int64

In [ ]:
# Q. What is the maximum SALE_PRICE in this dataset?
df['SALE_PRICE'].max()

260000000

In [ ]:
# Look at the row with the max SALE_PRICE
df[df['SALE_PRICE'] == 260000000]

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,NaN,D8,34 DESBROSSES STREET,NaN,10013.0,283.0,3.0,286.0,"36,858",305542.0,2007.0,2,D8,260000000,02/01/2019


In [ ]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?
df.TOTAL_UNITS.value_counts()

1.0      131
0.0       11
5.0        1
286.0      1
8.0        1
3.0        1
Name: TOTAL_UNITS, dtype: int64

In [ ]:
df[df['TOTAL_UNITS'] > 1.0]

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
1542,1,TRIBECA,02 TWO FAMILY DWELLINGS,1,132,26,NaN,S2,"75 WARREN STREET, XX",NaN,10007.0,2.0,1.0,3.0,"1,819",10117.0,1905.0,1,S2,12950000,01/10/2019
4002,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2B,143,16,NaN,D2,325 GREENWICH STREET,NaN,10013.0,4.0,4.0,8.0,"2,000",11726.0,1920.0,2,D2,7300000,01/23/2019
6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,NaN,D8,34 DESBROSSES STREET,NaN,10013.0,283.0,3.0,286.0,"36,858",305542.0,2007.0,2,D8,260000000,02/01/2019
8641,1,TRIBECA,07 RENTALS - WALKUP APARTMENTS,2A,193,38,NaN,C4,59 WALKER STREET,NaN,10013.0,4.0,1.0,5.0,"2,479",10311.0,1910.0,2,C4,0,02/13/2019


In [ ]:
# Keep only the single units
df= df[df['TOTAL_UNITS'] == 1.0]
df.shape

(131, 21)

In [ ]:
# Q. Now what is the max sales price? How many square feet does it have?
print(df.SALE_PRICE.max())
df[df.SALE_PRICE == df.SALE_PRICE.max()]

39285000


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
9236,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1144,NaN,R4,"70 VESTRY STREET, 12S",12S,10013.0,1.0,0.0,1.0,0,8346.0,2016.0,2,R4,39285000,02/15/2019


In [ ]:
df.loc[9236].GROSS_SQUARE_FEET

8346.0

In [ ]:
# Q. How often did $0 sales occur in this subset of the data?

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 

df.SALE_PRICE.value_counts()

0           15
36681561    15
2700000      2
4800000      2
4100000      2
            ..
1380000      1
7200000      1
3920263      1
1658000      1
9316988      1
Name: SALE_PRICE, Length: 98, dtype: int64

In [ ]:
len(df[df.SALE_PRICE == 0])

15

In [ ]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?
df[df.GROSS_SQUARE_FEET > 5000]

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
9236,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1144,NaN,R4,"70 VESTRY STREET, 12S",12S,10013.0,1.0,0.0,1.0,0,8346.0,2016.0,2,R4,39285000,02/15/2019
10487,1,TRIBECA,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,179,1607,NaN,R1,"24 LEONARD ST, PH W",PH W,10013.0,1.0,0.0,1.0,"9,983",39567.0,1940.0,2,R1,9857100,02/22/2019
16008,1,TRIBECA,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,179,1602,NaN,R1,"24 LEONARD STREET, 2",2,10013.0,1.0,0.0,1.0,"9,983",39567.0,1940.0,2,R1,9718000,03/21/2019


In [ ]:
df.GROSS_SQUARE_FEET.max()

39567.0

In [ ]:
# What are the building class categories?
# How frequently does each occur?
df.BUILDING_CLASS_CATEGORY.value_counts(ascending= False)

13 CONDOS - ELEVATOR APARTMENTS               121
15 CONDOS - 2-10 UNIT RESIDENTIAL               8
46 CONDO STORE BUILDINGS                        1
16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT      1
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [ ]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments

# Check how many rows you have now. (Should be 106 rows.)
df= df[(df.BUILDING_CLASS_CATEGORY == '13 CONDOS - ELEVATOR APARTMENTS') & (df.SALE_PRICE > 0)]

In [ ]:
df.shape

(106, 21)

In [ ]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE

import plotly.express as px
fig= px.scatter(x= df['GROSS_SQUARE_FEET'], y= df['SALE_PRICE'])
fig.show()

In [ ]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"

fig= px.scatter(df, x= 'GROSS_SQUARE_FEET', y= 'SALE_PRICE', trendline= 'ols')
fig.show()

In [ ]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?
high_price= df[df.SALE_PRICE > 35000000]
high_price['SALE_PRICE'].value_counts()

36681561    15
39285000     1
Name: SALE_PRICE, dtype: int64

In [ ]:
high_price.nunique()

BOROUGH                            1
NEIGHBORHOOD                       1
BUILDING_CLASS_CATEGORY            1
TAX_CLASS_AT_PRESENT               1
BLOCK                              1
LOT                               16
EASE-MENT                          0
BUILDING_CLASS_AT_PRESENT          1
ADDRESS                           16
APARTMENT_NUMBER                  16
ZIP_CODE                           1
RESIDENTIAL_UNITS                  1
COMMERCIAL_UNITS                   1
TOTAL_UNITS                        1
LAND_SQUARE_FEET                   1
GROSS_SQUARE_FEET                 10
YEAR_BUILT                         1
TAX_CLASS_AT_TIME_OF_SALE          1
BUILDING_CLASS_AT_TIME_OF_SALE     1
SALE_PRICE                         2
SALE_DATE                          2
dtype: int64

No, it's not dirty data because the units are (almost) all the same sales date and price, but they are on different lots, with different addresses and different square footage. So they sold 15 units at one address to some dude.

In [ ]:
# Make a judgment call:
# Keep rows where sale price was < $35 million

# Check how many rows you have now. (Should be down to 90 rows.)
df= df[df.SALE_PRICE < 35000000]
df.shape

(90, 21)

In [ ]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline
fig= px.scatter(df, x= 'GROSS_SQUARE_FEET', y= 'SALE_PRICE', trendline= 'ols')
fig.show()

In [ ]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,2800000,01/03/2019
763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,2650000,01/07/2019
1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,1005000,01/09/2019
1543,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,3624,NaN,R4,"2 RIVER TERRACE, 4C",4C,0.0,1.0,0.0,1.0,0,1088.0,2006.0,2,R4,1678000,01/10/2019
1855,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,2610,NaN,R4,"380 RECTOR PLACE, 10A",10A,10280.0,1.0,0.0,1.0,0,873.0,0.0,2,R4,1380000,01/11/2019


In [ ]:
df= df[['GROSS_SQUARE_FEET', 'SALE_PRICE']]

In [70]:
df.to_csv('tribeca.csv', index= False)